In [9]:
#import json
import ast

#Find the SCC around a given pivot state
#by finding an intersection of Forwards and Backwards reachability from pivot

def intersect_FB( visitedF, visitedB ):
    result = {} #will be stored as dictionary {rectangle : {entryF facet : list exitF facets}} in the forward direction
    for rkey in visitedF:
        if rkey in visitedB:
            ffacets = visitedF[ rkey ] # entry-> exits forward
            bfacets = visitedB[ rkey ] # exit -> entries from fw point of view
            rresult = { }# facet:list of exits for this given rectangle
            #compare, store the "intersection"
            r_intersectionF = {} #in the form of entry facet F : list of exit facets F
            for fkey in ffacets:
                #if fkey somewhere in list of some bfkey
                for bfkey in ffacets[ fkey ]: #forward [] of exits
                    if bfkey in bfacets: #is exit listed in bacwards?
                        if fkey in bfacets[ bfkey ]: #does exit backwards lead to fw entry?
                            #store
                            if fkey not in r_intersectionF:
                                r_intersectionF[ fkey ] = []
                            r_intersectionF[ fkey ].append( bfkey )
            result[ rkey ] = r_intersectionF #for rectangle r the intersection of fw and bw flow between facets
    return result

def nonempty_intersection( tiles1, tiles2 ):
    return ( max(tiles1[0],tiles2[0]) <=  min(tiles1[1],tiles2[1]) )

#makes sense only for nonempty intersection
def get_intersection( tiles1, tiles2 ):
    if nonempty_intersection( tiles1, tiles2 ):
        return [ max( tiles1[0], tiles2[0] ), min( tiles1[1], tiles2[1] ) ]

####here nuanced entry exit tiles todo...
def intersect_nuanced_tiles_FB( visitedF, visitedB ):
    result = {} #will be stored as dictionary {rectangle : {entryF facet : list exitF facets}} in the forward direction
    for rkey in visitedF:
        if rkey in visitedB:
            ffacets = visitedF[ rkey ] # entry-> exits forward
            bfacets = visitedB[ rkey ] # exit -> entries from fw point of view
            rresult = { }# facet:list of exits for this given rectangle
            #compare, store the "intersection"
            r_intersectionF = {} #in the form of entry facet F : list of exit facets F
            for fkey in ffacets:
                #if fkey somewhere in list of some bfkey
                for bfkey in ffacets[ fkey ]: #forward [] of exits
                    if bfkey in bfacets: #is exit listed in bacwards?
                        if fkey in bfacets[ bfkey ]: #does exit backwards lead to fw entry?
                            #store
                            if fkey not in r_intersectionF:
                                r_intersectionF[ fkey ] = []
                            r_intersectionF[ fkey ].append( bfkey )
            result[ rkey ] = r_intersectionF #for rectangle r the intersection of fw and bw flow between facets
    return result

def filter_with_entry_exit_pairs( dictionary ):
    result = {}
    for key in dictionary:
        if dictionary[ key ] != {}:
            result[ key ] = dictionary[ key ]
    return result

def write_out_inter( dictionar, i ):
    #write content to the output file
    fo = open("./outVDPinter/output"+str(i)+".txt", "w")
    #fo.write("Intersection of forward and backward reachable states:\n")
    fo.write( str(dictionar) )
    fo.close()

def read_dict_from_file( fb, i ):
    if fb == "f":
        fo = open("./outVDPforward/output"+str(i)+".txt", "r")
    else:
        fo = open("./outVDPbackward/output"+str(i)+".txt", "r")
    data = fo.read()
    fo.close()
    #js = json.loads(data)
    #with_double_quotes = data.replace( "\'", '"' )
    dictionary = ast.literal_eval( data ) #with_double_quotes )
    return dictionary


In [5]:
#F and B of point A [49,66] facets 00<->11
#computedPartialB = "4000Q97FROM_57,71_10_0K"
#computedPartialF = ""
#computedLast = "Konec"
visitedF = read_dict_from_file( "f", "Konec"+"FROM_57,71_00_0K" )
visitedB = read_dict_from_file( "b", "Konec"+"FROM_57,71_10_0K" )
intersect = intersect_FB( visitedF, visitedB )
write_out_inter( intersect, "FROM_57,71_0010_intersect_rectangles_only" )
intersectFiltered = filter_with_entry_exit_pairs( intersect )
write_out_inter( intersectFiltered, "FROM_57,71_0010_intersect_entry_exit" )

FileNotFoundError: [Errno 2] No such file or directory: './outVDPforward/outputKonecFROM_57,71_00_0K.txt'

In [6]:
#results for whole facets - mimic the behaviour of FA
visitedF = read_dict_from_file( "f", "Konec"+"FROM_57,71_00_wholefacets" )
visitedB = read_dict_from_file( "b", "Konec"+"FROM_57,71_10_wholefacets" )
intersect = intersect_FB( visitedF, visitedB )
write_out_inter( intersect, "FROM_57,71_0010_intersect_rectangles_only_wholefacets" )
intersectFiltered = filter_with_entry_exit_pairs( intersect )
write_out_inter( intersectFiltered, "FROM_57,71_0010_intersect_entry_exit_wholefacets" )

In [7]:
#visualize the sets, write png images
#need the tresholds and rectangle->geom real
#import
import numpy as np
import seaborn as sns
# Matplotlib for additional customization
from matplotlib import pyplot as plt
%matplotlib inline

import pandas as pd

def r_hash( h ):
    r = [0,0] #default value
    hparts = h.split('|')
    r[0] = int( hparts[0] )
    r[1] = int( hparts[1] )
    return r

#model-specific tresholds
x0min = -8
x0max = 8
x1min = -8
x1max = 8

pieces_count = 100

stepx0 = ( x0max - x0min ) / pieces_count
stepx1 = ( x1max - x1min ) / pieces_count

tresholdsX0 = np.arange ( x0min, x0max+(0.3*stepx0), stepx0 )
tresholdsX1 = np.arange ( x1min, x1max+(0.3*stepx1), stepx1 )

tresholds = [ tresholdsX0, tresholdsX1 ]

def visualize_rectangles( rdict, i ):
    list_r = []
    
    for key in rdict:
        r = r_hash( key )
        list_r.append( r )
    
    xlist = []
    ylist = []
    for r in list_r:
        xlist.append( 0.5*( tresholds[0][r[0]] + tresholds[0][r[0]+1]) )
        ylist.append( 0.5*( tresholds[1][r[1]] + tresholds[1][r[1]+1]) )

    ctverecky = pd.DataFrame({
        "x" : xlist,
        "y" : ylist
    })

    wid = 5
    hei = 5
    fig,ax = plt.subplots(1, 1, figsize=(wid, hei))
    sns.scatterplot(x="x", y="y", data=ctverecky, markers="s", s=10)
    ax.set(xlim=(x0min, x0max))
    ax.set(ylim=(x1min, x1max))
    plt.savefig('./outVDPinter/img'+str(i)+'.png', dpi=300)
    plt.clf()
    plt.close()

In [8]:
visualize_rectangles( visitedF, "Forward" )
visualize_rectangles( visitedB, "Backward" )
visualize_rectangles( intersect, "Intersect" )
visualize_rectangles( intersectFiltered, "IntersectEE" )
